In [ ]:
import pandas as pd
import numpy as np

In [1]:
# Чтение Excel-файла, замена точек на запятые в столбце CLOSE и сохранение результата в новый файл

file_name = "index_values.xlsx" 
df = pd.read_excel(file_name)

df['CLOSE'] = df['CLOSE'].astype(str).str.replace('.', ',', regex=False)

df.to_excel("updated_index_values.xlsx", index=False)

print("Файл обновлен и сохранен как 'updated_index_values.xlsx'")

Файл обновлен и сохранен как 'updated_index_values.xlsx'


In [46]:
# Чтение Excel-файла с котировками, преобразование даты в индекс, расчёт месячных доходностей и сохранение результата в новый файл

file_name = "котировоки_драг_металлы.xlsx"
sheet_name = "Котировки" 
df = pd.read_excel(file_name, sheet_name=sheet_name)\

df['Дата'] = pd.to_datetime(df['Дата'], format='%Y-%m-%d')

df.set_index('Дата', inplace=True)

monthly_returns = df.resample('M').last().pct_change().dropna() * 100

monthly_returns.reset_index(inplace=True)

monthly_returns.to_excel("monthly_metall_returns.xlsx", index=False)

print("Месячные доходности сохранены в 'monthly_returns.xlsx'")

Месячные доходности сохранены в 'monthly_returns.xlsx'


In [72]:
# Чтение данных доходностей, расчёт ковариационной матрицы и её нормализованной версии, сохранение обеих в Excel

input_file = 'oblig_weekly_returns.xlsx'
output_file = 'ковариации_oblig_weekly.xlsx'

data = pd.read_excel(input_file)

data_without_dates = data.iloc[:, 1:]

cov_matrix = data_without_dates.cov()

min_val = cov_matrix.min().min()
max_val = cov_matrix.max().max() 
normalized_matrix = (cov_matrix - min_val) / (max_val - min_val)

with pd.ExcelWriter(output_file) as writer:
    cov_matrix.to_excel(writer, sheet_name='Ковариации')
    normalized_matrix.to_excel(writer, sheet_name='Нормализованная')

print(f"Матрицы сохранены в файл {output_file}")

Матрицы сохранены в файл ковариации_oblig_weekly.xlsx


In [48]:
# Чтение ковариационной матрицы и весов из Excel, расчёт дисперсии портфеля по формуле wᵀΣw и вывод результата

import pandas as pd
import numpy as np

file_path = 'Финальная_сборка_по_акциям.xlsx'
sheet_name = 'Подсчет_дисперсии'

data = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

cov_matrix = data.iloc[5:21, 3:21].values 
# print(cov_matrix)
tickers_matrix = data.iloc[4, 3:21].values
# print(tickers_matrix)
tickers_column = data.iloc[5:21, 2].values
# print(tickers_column)

weights = cov_matrix[0:16, 16]
# print(weights)

portfolio_variance = 0
n = len(weights)

for i in range(n):
    for j in range(n):
        portfolio_variance += weights[i] * weights[j] * cov_matrix[i, j]
        # print(weights[i], weights[j], cov_matrix[i, j])

print(f"Дисперсия портфеля: {portfolio_variance}")


Дисперсия портфеля: 46.9576320114609


In [50]:
# Чтение данных доходностей, расчёт корреляций золота и серебра с инфляцией, а также средней разницы и стандартного отклонения между ними и инфляцией

file_path = 'monthly_metall_returns.xlsx'

data = pd.read_excel(file_path)

correlation = data['GLDRUB_TOM'].corr(data['SLVRUB_TOM'])
print(f"Корреляция между GLDRUB_TOM и SLVRUB_TOM: {correlation:.4f}")
correlation_1 = data['GLDRUB_TOM'].corr(data['Прирост инфляции'])
print(f"Корреляция между GLDRUB_TOM и Прирост инфляции: {correlation_1:.4f}")
correlation_2 = data['SLVRUB_TOM'].corr(data['Прирост инфляции'])
print(f"Корреляция между SLVRUB_TOM и Прирост инфляции: {correlation_2:.4f}")

gld_infl_diff = data['GLDRUB_TOM'] - data['Прирост инфляции']
gld_infl_mean_diff = gld_infl_diff.mean()
gld_infl_std_diff = gld_infl_diff.std()
print(f"Средняя разница GLDRUB_TOM и Прирост инфляции: {gld_infl_mean_diff:.4f}")
print(f"Стандартное отклонение для этой разницы: {gld_infl_std_diff:.4f}")

slv_infl_diff = data['SLVRUB_TOM'] - data['Прирост инфляции']
slv_infl_mean_diff = slv_infl_diff.mean()
slv_infl_std_diff = slv_infl_diff.std()
print(f"Средняя разница SLVRUB_TOM и Прирост инфляции: {slv_infl_mean_diff:.4f}")
print(f"Стандартное отклонение для этой разницы: {slv_infl_std_diff:.4f}")


Корреляция между GLDRUB_TOM и SLVRUB_TOM: 0.5328
Корреляция между GLDRUB_TOM и Прирост инфляции: -0.2466
Корреляция между SLVRUB_TOM и Прирост инфляции: 0.0595
Средняя разница GLDRUB_TOM и Прирост инфляции: 1.4002
Стандартное отклонение для этой разницы: 8.7931
Средняя разница SLVRUB_TOM и Прирост инфляции: 2.1805
Стандартное отклонение для этой разницы: 11.2132


In [2]:
# Подключение к API Tinkoff Invest, поиск инструмента по тикеру "POLY" и вывод его тикера, ID и названия

import os
from tinkoff.invest import Client

TOKEN = "YOUR_TOKEN_HERE" 

def main():
    with Client(TOKEN) as client:
        response = client.instruments.find_instrument(query="POLY")
        for instrument in response.instruments:
            print(f"Ticker: {instrument.ticker}, ID: {instrument.uid}, Name: {instrument.name}")

if __name__ == "__main__":
    main()


Ticker: POH3, ID: 74d48089-ace3-42c8-a977-63f357f1bc2b, Name: POLY-3.23 Polymetal
Ticker: POZ2, ID: 33b5341a-3bf4-4845-bb5c-c94037e4bedf, Name: POLY-12.22 Polymetal
Ticker: POH2, ID: 1f98e432-d5db-4caf-a172-80985f0f1ae9, Name: POLY-3.22 Polymetal
Ticker: 1066, ID: 9075043a-9de1-421a-8225-381a485d0ea2, Name: Shandong Weigao Group Medical Polymer
Ticker: POM2, ID: 1f99a2bd-1ff2-4541-ac82-58c550892156, Name: POLY-6.22 Polymetal
Ticker: POZ3, ID: 53f564be-671e-4f9d-8dfa-4fdeb27dc2f3, Name: POLY-12.23 Polymetal
Ticker: POH4, ID: 307bdee0-639f-4e31-98de-d815c2af049b, Name: POLY-3.24 Polymetal
Ticker: POLY, ID: 127361c2-32ec-448c-b3ec-602166f537ea, Name: Solidcore Resources plc
Ticker: POZ1, ID: 1f9722e5-3afa-4904-b21b-c4d822f8e60e, Name: POLY-12.21 Polymetal
Ticker: POU4, ID: fc4d7da2-626b-4ba5-aa22-e9dea1ed3858, Name: POLY-9.24 Solidcore
Ticker: POU2, ID: 2502142d-06ec-44a7-b382-861b9a8b6053, Name: POLY-9.22 Polymetal
Ticker: PLZL@GS, ID: fc80c408-157a-4c00-b437-05e23b646d91, Name: Polyus P

In [5]:
# Подключение к API Tinkoff Invest, загрузка месячных свечей за ~5000 дней по инструменту и вывод дат с ценами закрытия

from datetime import timedelta
from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now

TOKEN = "YOUR_TOKEN_HERE" 

def main():
    print("Запуск программы...")
    
    with Client(TOKEN) as client:
        print("Подключение к API...")
        
        instrument_id = "127361c2-32ec-448c-b3ec-602166f537ea" 

        candles = client.get_all_candles(
            instrument_id=instrument_id,
            from_=now() - timedelta(days=5000),
            interval=CandleInterval.CANDLE_INTERVAL_MONTH,
        )
        
        print("Данные получены, обработка...")
        for candle in candles:
            date = candle.time.strftime("%Y-%m")
            close_price = candle.close.units + candle.close.nano / 1e9
            
            print(f"\nДанные за месяц: {date}")
            print(f"  Закрытие (цена облигации): {close_price:.2f}")

    print("Программа завершена.")

if __name__ == "__main__":
    main()


Запуск программы...
Подключение к API...
Данные получены, обработка...

Данные за месяц: 2013-06
  Закрытие (цена облигации): 225.00

Данные за месяц: 2013-07
  Закрытие (цена облигации): 322.60

Данные за месяц: 2013-08
  Закрытие (цена облигации): 392.40

Данные за месяц: 2013-09
  Закрытие (цена облигации): 345.00

Данные за месяц: 2013-10
  Закрытие (цена облигации): 308.70

Данные за месяц: 2013-11
  Закрытие (цена облигации): 291.20

Данные за месяц: 2013-12
  Закрытие (цена облигации): 309.80

Данные за месяц: 2014-01
  Закрытие (цена облигации): 333.20

Данные за месяц: 2014-02
  Закрытие (цена облигации): 381.20

Данные за месяц: 2014-03
  Закрытие (цена облигации): 367.50

Данные за месяц: 2014-04
  Закрытие (цена облигации): 335.10

Данные за месяц: 2014-05
  Закрытие (цена облигации): 309.00

Данные за месяц: 2014-06
  Закрытие (цена облигации): 328.10

Данные за месяц: 2014-07
  Закрытие (цена облигации): 307.20

Данные за месяц: 2014-08
  Закрытие (цена облигации): 327.30

In [64]:
# Получение дневных котировок для заданных облигаций через API Tinkoff Invest, объединение в один DataFrame и сохранение/обновление Excel-файла

import os
from datetime import timedelta
import pandas as pd
from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now

TOKEN = "YOUR_TOKEN_HERE"  # Вставьте ваш токен API
EXCEL_FILE = "котировоки_облигации.xlsx"

# Словарь с тикерами и их идентификаторами
# assets = {"UPRO": "664921c5-b552-47a6-9ced-8735a3c6ca8a", "VKCO": "b71bd174-c72c-41b0-a66f-5f9073e0d1f5", 
#         "OZON": "35fb8d6b-ed5f-45ca-b383-c4e3752c9a8a", "SVCB": "1fbecbbc-ef32-448c-b4fe-b0037795ba01",
#         "YDEX": "7de75794-a27f-4d81-a39b-492345813822", "KMAZ": "f8f1de0d-5a5f-47e9-8ecb-00bec8f8351f",
#         "AFLT":"1c69e020-f3b1-455c-affa-45f8b8049234", "RTKMP": "e1b089f3-9bf1-44c3-897f-25e9f591bebc",
#         "CHMF": "fa6aae10-b8d5-48c8-bbfd-d320d925d096", "MAGN": "7132b1c9-ee26-4464-b5b5-1046264b61d9", 
#         "GAZP": "962e2a95-02a9-4171-abd7-aa198dbe643a", "SBER": "e6123145-9665-43e0-8413-cd61b8aa9b13",
#         "ROSN": "fd417230-19cf-4e7b-9623-f7c9ca18ec6b", "NLMK": "161eb0d0-aaac-4451-b374-f5d0eeb1b508",
#         "LKOH": "02cfdf61-6298-4c0f-a9ca-9cabc82afaf3", "VTBR": "8e2b0325-0292-4654-8a18-4f63ed3b0e09",
#         "GMKN": "509edd0c-129c-4ee2-934d-7f6246126da1"
#        }

# assets = {"GLDRUB_TOM": "258e2b93-54e8-4f2d-ba3d-a507c47e3ae2", 
#          "SLVRUB_TOM": "d6240afe-4e9c-49b6-8835-629f431c8506"}

assets = {"SU26212RMFS9": "c5711a61-94c5-4c68-852b-6122fdac79a4",
          "RU000A1069P3": "cf331b84-b924-48f3-8878-4643047d5946", 
          "RU000A1017J5": "bb14e018-6bfc-41ec-b9ab-96c57679f793", 
          "RU000A101FH6": "2478e8ad-5104-4b23-b9c2-cf275a0f3ac8", 
          "RU000A1007Z2": "eaddb925-5d1a-4d02-9d70-f846937ae4ab", 
          "SU26244RMFS2": "84842c97-75a2-4ae6-9be6-39e5d93a0ca9", 
         }

def fetch_asset_data(client, ticker, instrument_id):
    """
    Получение данных свечей для указанного тикера.
    """
    print(f"Получение данных для {ticker}...")
    candles = client.get_all_candles(
        instrument_id=instrument_id,
        from_=now() - timedelta(days=10000),
        interval=CandleInterval.CANDLE_INTERVAL_DAY,
    )
    data = {
        "Дата": [],
        ticker: [],
    }
    for candle in candles:
        date = candle.time.strftime("%Y-%m-%d")  # Год-месяц
        close_price = candle.close.units + candle.close.nano / 1e9
        data["Дата"].append(date)
        data[ticker].append(close_price)
    return pd.DataFrame(data)

def main():
    print("Запуск программы...")  # Сообщение о начале выполнения
    
    with Client(TOKEN) as client:
        all_dataframes = []  # Список для объединения всех данных
        
        for ticker, instrument_id in assets.items():
            df = fetch_asset_data(client, ticker, instrument_id)
            all_dataframes.append(df)
        
        # Объединяем все данные в один DataFrame
        df_combined = pd.concat(all_dataframes, axis=1)
        df_combined = df_combined.loc[:, ~df_combined.columns.duplicated()]  # Удаляем дубликаты столбцов "Дата"
        
        if os.path.exists(EXCEL_FILE):
            print(f"Файл {EXCEL_FILE} найден. Добавляем данные...")
            df_existing = pd.read_excel(EXCEL_FILE)
            df_result = pd.merge(df_existing, df_combined, on="Дата", how="outer")
        else:
            print(f"Файл {EXCEL_FILE} не найден. Создаем новый файл...")
            df_result = df_combined
        
        # Сохраняем в Excel
        df_result.to_excel(EXCEL_FILE, index=False)
        print(f"Данные успешно записаны в {EXCEL_FILE}.")

if __name__ == "__main__":
    main()


Запуск программы...
Получение данных для SU26212RMFS9...
Получение данных для RU000A1069P3...
Получение данных для RU000A1017J5...
Получение данных для RU000A101FH6...
Получение данных для RU000A1007Z2...
Получение данных для SU26244RMFS2...
Файл котировоки_облигации.xlsx не найден. Создаем новый файл...
Данные успешно записаны в котировоки_облигации.xlsx.


In [85]:
# Чтение Excel-файла с данными облигаций, преобразование дат в индекс, расчёт недельных доходностей и сохранение результата в новый файл

import pandas as pd

file_name = "Дисперсия_облигаций.xlsx" 
sheet_name = "Лист1"
df = pd.read_excel(file_name, sheet_name=sheet_name)\

df['Дата'] = pd.to_datetime(df['Дата'], format='%Y-%m-%d')

df.set_index('Дата', inplace=True)

monthly_returns = df.resample('W').last().pct_change().dropna() * 100

monthly_returns.reset_index(inplace=True)

monthly_returns.to_excel("oblig_weekly_returns.xlsx", index=False)

print("Месячные доходности сохранены в 'oblig_weekly_returns.xlsx'")


Месячные доходности сохранены в 'oblig_weekly_returns.xlsx'


/var/folders/cr/kkk6yb8j5ml02kjf2pmnw19w0000gn/T/ipykernel_93107/2311107535.py:15: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  monthly_returns = df.resample('W').last().pct_change().dropna() * 100


In [ ]:
# Оптимизация портфеля по годам (SciPy): считаем веса с макс. Шарпом и с мин. волатильностью при целевой доходности, выводим вторые

# import numpy as np
import pandas as pd
from scipy.optimize import minimize

data = {
    "Годы": list(range(2025, 2045)),
    "Облигации": [
        20.64, 20.64, 19.51, 18.12, 17.50, 15.06, 14.49, 14.48, 14.33, 14.31, 
        13.75, 13.75, 13.49, 13.50, 13.49, 13.58, 13.64, 13.65, 13.46, 13.45
    ],
    "Акции": [23.07] * 20,
    "Депозит": [21.59, 21.59, 21.59, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
}

volatilities = {
    "Облигации": 0.64 / 100,
    "Акции": 6.85 / 100,
    "Депозит": 0.0 / 100
}

returns_df = pd.DataFrame(data)
assets = ["Облигации", "Акции", "Депозит"]
volatilities_vector = np.array([volatilities[asset] for asset in assets])

def portfolio_performance(weights, returns, volatilities):
    weights = np.array(weights)
    portfolio_return = np.sum(weights * returns)
    portfolio_volatility = np.sqrt(np.sum((weights * volatilities) ** 2))
    return portfolio_return, portfolio_volatility

def sharpe_ratio(weights, returns, volatilities, risk_free_rate=0.0):
    portfolio_return, portfolio_volatility = portfolio_performance(weights, returns, volatilities)
    return (portfolio_return - risk_free_rate) / portfolio_volatility

def optimize_portfolio(year_returns, volatilities, objective="sharpe", target_return=None):
    num_assets = len(year_returns)
    bounds = [(0, 1) for _ in range(num_assets)]
    constraints = [{'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}]
    
    if objective == "sharpe":
        result = minimize(
            lambda weights: -sharpe_ratio(weights, year_returns, volatilities),
            x0=np.array([1/num_assets] * num_assets),
            bounds=bounds,
            constraints=constraints
        )
    elif objective == "min_vol":
        constraints.append({'type': 'ineq', 'fun': lambda weights: np.sum(weights * year_returns) - target_return})
        result = minimize(
            lambda weights: portfolio_performance(weights, year_returns, volatilities)[1],
            x0=np.array([1/num_assets] * num_assets),
            bounds=bounds,
            constraints=constraints
        )
        
    optimal_weights = np.round(result.x, 2)
    optimal_weights /= np.sum(optimal_weights)
    return optimal_weights

results_sharpe = []
results_min_vol = []

for index, row in returns_df.iterrows():
    year = row["Годы"]
    year_returns = row[assets].values / 100 
    weights_sharpe = optimize_portfolio(year_returns, volatilities_vector, objective="sharpe")
    results_sharpe.append((year, weights_sharpe))
    target_return = max(year_returns) * 0.5775
    weights_min_vol = optimize_portfolio(year_returns, volatilities_vector, objective="min_vol", target_return=target_return)
    results_min_vol.append((year, weights_min_vol))

# Вывод результатов
# print("Оптимальные веса для максимального коэффициента Шарпа:")
# for year, weights in results_sharpe:
#     print(f"Год {year}: {dict(zip(assets, weights))}")

print("\nОптимальные веса для минимальной волатильности с высокой доходностью:")
for year, weights in results_min_vol:
    print(f"Год {year}: {dict(zip(assets, weights))}")


In [102]:
# Оптимизация по годам: при заданных доходностях и волатильностях считаем портфель минимальной волатильности с доходностью ≥ 85% от максимальной и печатаем веса

import numpy as np
import pandas as pd
from scipy.optimize import minimize

data = {
    "Годы": list(range(2025, 2045)),
    "Облигации": [
        20.64, 20.64, 19.51, 18.12, 17.50, 15.06, 14.49, 14.48, 14.33, 14.31, 
        13.75, 13.75, 13.49, 13.50, 13.49, 13.58, 13.64, 13.65, 13.46, 13.45
    ],
    "Акции": [23.07] * 20,
    "Депозит": [21.59, 21.59, 21.59, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
}

volatilities = {
    "Облигации": 0.64 / 100,
    "Акции": 6.85 / 100,
    "Депозит": 0.01 / 100 
}

returns_df = pd.DataFrame(data)
assets = ["Облигации", "Акции", "Депозит"]
volatilities_vector = np.array([volatilities[asset] for asset in assets])

def portfolio_performance(weights, returns, volatilities):
    weights = np.array(weights)
    portfolio_return = np.sum(weights * returns)
    portfolio_volatility = np.sqrt(np.sum((weights * volatilities) ** 2))
    return portfolio_return, portfolio_volatility

def optimize_portfolio(year_returns, volatilities, target_return):
    num_assets = len(year_returns)
    bounds = [(0, 1) for _ in range(num_assets)]
    constraints = [
        {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Сумма весов равна 1
        {'type': 'ineq', 'fun': lambda weights: np.sum(weights * year_returns) - target_return}  # Доходность >= целевой
    ]
    
    result = minimize(
        lambda weights: portfolio_performance(weights, year_returns, volatilities)[1],
        x0=np.array([1/num_assets] * num_assets),
        bounds=bounds,
        constraints=constraints
    )
    
    optimal_weights = np.round(result.x, 2)
    optimal_weights /= np.sum(optimal_weights)
    return optimal_weights

results_risky_portfolios = []

for index, row in returns_df.iterrows():
    year = row["Годы"]
    year_returns = row[assets].values / 100 
    max_return = max(year_returns) 
    target_return = max_return * 0.85 
    weights_risky = optimize_portfolio(year_returns, volatilities_vector, target_return)
    results_risky_portfolios.append((year, weights_risky))

print("Рисковый портфель с доходностью > 85% от максимальной:")
for year, weights in results_risky_portfolios:
    print(f"Год {year}: {dict(zip(assets, weights))}")


Рисковый портфель с доходностью > 85% от максимальной:
Год 2025.0: {'Облигации': 0.0, 'Акции': 0.0, 'Депозит': 1.0}
Год 2026.0: {'Облигации': 0.0, 'Акции': 0.0, 'Депозит': 1.0}
Год 2027.0: {'Облигации': 0.0, 'Акции': 0.0, 'Депозит': 1.0}
Год 2028.0: {'Облигации': 0.7, 'Акции': 0.3, 'Депозит': 0.0}
Год 2029.0: {'Облигации': 0.62, 'Акции': 0.38, 'Депозит': 0.0}
Год 2030.0: {'Облигации': 0.43, 'Акции': 0.57, 'Депозит': 0.0}
Год 2031.0: {'Облигации': 0.4, 'Акции': 0.6, 'Депозит': 0.0}
Год 2032.0: {'Облигации': 0.4, 'Акции': 0.6, 'Депозит': 0.0}
Год 2033.0: {'Облигации': 0.4, 'Акции': 0.6, 'Депозит': 0.0}
Год 2034.0: {'Облигации': 0.4, 'Акции': 0.6, 'Депозит': 0.0}
Год 2035.0: {'Облигации': 0.37, 'Акции': 0.63, 'Депозит': 0.0}
Год 2036.0: {'Облигации': 0.37, 'Акции': 0.63, 'Депозит': 0.0}
Год 2037.0: {'Облигации': 0.36, 'Акции': 0.64, 'Депозит': 0.0}
Год 2038.0: {'Облигации': 0.36, 'Акции': 0.64, 'Депозит': 0.0}
Год 2039.0: {'Облигации': 0.36, 'Акции': 0.64, 'Депозит': 0.0}
Год 2040.0: {'Об